In [0]:
%pip install --upgrade databricks-sdk==0.90.0
%restart_python

In [0]:
%sh apt-get update && apt-get install -y gdal-bin libgdal-dev postgis

In [0]:
import os
import subprocess
import glob

# Local temp directory on the driver
local_dir = "/tmp/georaster_demo"
os.makedirs(local_dir, exist_ok=True)

srid = "27700"                 # adjust if using a different GeoTIFF
catalog = "catalog_stuarlqxck_tcgppf"
lakehouse_schema = "raster"
imagery_volume = "imagery"

os.environ["CATALOG"] = catalog
os.environ["SCHEMA"] = lakehouse_schema
os.environ["PLUGIN_VOLUME"] = "ecw"

lakebase_schema = "raster"            # your Lakebase schema
table = "imagery"       # table to create
output_sql = os.path.join(local_dir, f"{table}.sql")

rasters = glob.glob(f"/Volumes/{catalog}/{lakehouse_schema}/{imagery_volume}/*.ecw", recursive=True)

In [0]:
%sh
cp /Volumes/$CATALOG/$SCHEMA/$PLUGIN_VOLUME/gdal_ECW_JP2ECW.so $(gdal-config --plugindir)

In [0]:
%sh
export CPL_DEBUG=ON
gdalinfo --formats

In [0]:
%sh which raster2pgsql

In [0]:
%sh raster2pgsql

In [0]:
%sh raster2pgsql -G

In [0]:
# Example tiling and options – change as desired
# -I: create GiST index, -C: apply raster constraints
# -M: vacuum analyze, -F: add a raster column named "rast"
# -l: add overviews for fast rendering in GIS platforms
# -t 512x512: tile size
cmd = [
    "raster2pgsql",
    "-I",
    "-C",
    "-d",
    "-e",
    "-M",
    "-F",
    "-l", "2,4,8",
    "-t", "512x512",
    "-s", srid,
    *rasters,
    f"{lakebase_schema}.{table}"
]

print("Running:", " ".join(cmd))

with open(output_sql, "w") as f:
    subprocess.run(cmd, check=True, stdout=f)

print("Generated SQL at:", output_sql)

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.postgres import RequestedClaims, RequestedResource

w = WorkspaceClient()
project_id = "61ad7f1c-95f4-4143-8b66-000942afcb89"
branch_id = "br-odd-silence-e1b8my4j"
endpoint_id = "ep-blue-mouse-e1jj0xrm"
endpoint = f"projects/{project_id}/branches/{branch_id}/endpoints/{endpoint_id}"
creds = w.postgres.generate_database_credential(endpoint)

In [0]:
os.environ["PGPASSWORD"] = creds.token

+ create database raster
+ create schema raster
+ install postgis and postgis_raster extensions

In [0]:
%sh psql -h 'ep-blue-mouse-e1jj0xrm.database.eastus2.azuredatabricks.net' --set=sslmode=require -d 'raster' -U 'stuart.lynn@databricks.com' -f /tmp/georaster_demo/imagery.sql